#  Faiss

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("data/loreal_shareholder_2022.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


Created a chunk of size 1024, which is longer than the specified 1000


In [5]:
len(docs)

2

In [7]:
embeddings=OllamaEmbeddings(model="llama3.2:1b")
db=FAISS.from_documents(docs,embeddings)
db

In [8]:
### querying 
query="How much loyalty is given to shareholders?"
docs=db.similarity_search(query)
docs[0].page_content


'“Dear Shareholders,\nL’Oréal continues on the path to success with an ever-stronger ambition, while acting with the sense of responsibility of a global leader. Dual financial and social excellence will always be at the heart of our business model.\nWe have set ourselves the ultimate goal of creating value that benefits everyone.\nWe create value for you, our shareholders.\xa0The resilience and outperformance of your Company are the perfect demonstration of its robust, virtuous and value creating business model. The quality of our results puts us in a position to offer a dividend of €6 per share, representing a significant increase of +25%. And the preferential dividend with a 10% loyalty bonus(1), at €6.60, is recognition of your long-term loyalty.\nI also know that you attach just as much importance to the quality of our relationship with you, our shareholders. I am delighted to welcome the more than 30,000 new individual shareholders in France who joined us in 2022.\nWe create value

##  As a Retriever

We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [9]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'“Dear Shareholders,\nL’Oréal continues on the path to success with an ever-stronger ambition, while acting with the sense of responsibility of a global leader. Dual financial and social excellence will always be at the heart of our business model.\nWe have set ourselves the ultimate goal of creating value that benefits everyone.\nWe create value for you, our shareholders.\xa0The resilience and outperformance of your Company are the perfect demonstration of its robust, virtuous and value creating business model. The quality of our results puts us in a position to offer a dividend of €6 per share, representing a significant increase of +25%. And the preferential dividend with a 10% loyalty bonus(1), at €6.60, is recognition of your long-term loyalty.\nI also know that you attach just as much importance to the quality of our relationship with you, our shareholders. I am delighted to welcome the more than 30,000 new individual shareholders in France who joined us in 2022.\nWe create value

##  Similarity Search with score

There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [11]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='25f0de62-ec7f-4eb9-bf39-bedc7074d577', metadata={'source': 'data/loreal_shareholder_2022.txt'}, page_content='“Dear Shareholders,\nL’Oréal continues on the path to success with an ever-stronger ambition, while acting with the sense of responsibility of a global leader. Dual financial and social excellence will always be at the heart of our business model.\nWe have set ourselves the ultimate goal of creating value that benefits everyone.\nWe create value for you, our shareholders.\xa0The resilience and outperformance of your Company are the perfect demonstration of its robust, virtuous and value creating business model. The quality of our results puts us in a position to offer a dividend of €6 per share, representing a significant increase of +25%. And the preferential dividend with a 10% loyalty bonus(1), at €6.60, is recognition of your long-term loyalty.\nI also know that you attach just as much importance to the quality of our relationship with you, our shareholders. 

In [12]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[-0.881639838218689,
 2.750432252883911,
 0.024656707420945168,
 -0.12043123692274094,
 4.924970626831055,
 0.11764147877693176,
 0.7593784928321838,
 1.6582592725753784,
 1.8138335943222046,
 2.4005157947540283,
 -1.7269155979156494,
 -0.5991809964179993,
 1.7194464206695557,
 -0.9917807579040527,
 3.0606303215026855,
 -4.258486747741699,
 -4.399342060089111,
 -0.09007548540830612,
 6.247124195098877,
 1.5605639219284058,
 -5.1955366134643555,
 1.633870244026184,
 0.3272535800933838,
 0.7464815378189087,
 -0.6505718231201172,
 3.1386115550994873,
 -4.8225860595703125,
 0.9664703011512756,
 -1.8964807987213135,
 2.3793797492980957,
 1.9391436576843262,
 0.005563139449805021,
 0.886970579624176,
 2.5291025638580322,
 0.9946305751800537,
 2.309476137161255,
 0.8120403289794922,
 0.05539470165967941,
 -1.0087333917617798,
 1.6373947858810425,
 0.18522244691848755,
 -3.8686916828155518,
 -0.6867102980613708,
 3.1292243003845215,
 0.2649444341659546,
 5.474104881286621,
 4.1902923583984375,

In [14]:
docs=db.similarity_search_by_vector(embedding_vector)
docs

[Document(id='25f0de62-ec7f-4eb9-bf39-bedc7074d577', metadata={'source': 'data/loreal_shareholder_2022.txt'}, page_content='“Dear Shareholders,\nL’Oréal continues on the path to success with an ever-stronger ambition, while acting with the sense of responsibility of a global leader. Dual financial and social excellence will always be at the heart of our business model.\nWe have set ourselves the ultimate goal of creating value that benefits everyone.\nWe create value for you, our shareholders.\xa0The resilience and outperformance of your Company are the perfect demonstration of its robust, virtuous and value creating business model. The quality of our results puts us in a position to offer a dividend of €6 per share, representing a significant increase of +25%. And the preferential dividend with a 10% loyalty bonus(1), at €6.60, is recognition of your long-term loyalty.\nI also know that you attach just as much importance to the quality of our relationship with you, our shareholders. I

# Saving FAISS vector db locally

In [15]:
### Saving And Loading
db.save_local("faiss_index")

In [16]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [17]:
docs

[Document(id='25f0de62-ec7f-4eb9-bf39-bedc7074d577', metadata={'source': 'data/loreal_shareholder_2022.txt'}, page_content='“Dear Shareholders,\nL’Oréal continues on the path to success with an ever-stronger ambition, while acting with the sense of responsibility of a global leader. Dual financial and social excellence will always be at the heart of our business model.\nWe have set ourselves the ultimate goal of creating value that benefits everyone.\nWe create value for you, our shareholders.\xa0The resilience and outperformance of your Company are the perfect demonstration of its robust, virtuous and value creating business model. The quality of our results puts us in a position to offer a dividend of €6 per share, representing a significant increase of +25%. And the preferential dividend with a 10% loyalty bonus(1), at €6.60, is recognition of your long-term loyalty.\nI also know that you attach just as much importance to the quality of our relationship with you, our shareholders. I